In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

In [2]:
cid ='d888370f9c2a4514aade84a1b6ae976e'
secret ='01774fdc665e4dcc8dc8c48667e46bf1'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
from requests import exceptions
import lyricsgenius as lg
import re
import requests

token = '0GGXQLMawu2kxxYNqTGFT79I0wq8McYb8jbOz0HGOQc-sCAmu0HuYKo_C71dsRg-'
genius = lg.Genius(token, retries=3)
genius.verbose = True
# Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.remove_section_headers = True

# Include hits thought to be non-songs (e.g. track lists)
genius.skip_non_songs = True

# Exclude songs with these words in their title
genius.excluded_terms = ["(Remix)", "(Live)"]

In [4]:
l = ['7eX5SypK35V8Y9d9pS6rWy',
        '4fPBB44eDH71YohayI4eKV',
        #'2M7UdnD0fEaryh8TnCvqFX',
        #'6LqZFfv4fUP7va14Y6VW9a',
]

print(sp.audio_features(tracks=l))

np.random.seed(3407)
R = np.random.normal(0, 1, (25, 384))

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('msmarco-MiniLM-L-6-v3')

[{'danceability': 0.472, 'energy': 0.746, 'key': 11, 'loudness': -4.816, 'mode': 1, 'speechiness': 0.0321, 'acousticness': 0.00581, 'instrumentalness': 0.663, 'liveness': 0.0999, 'valence': 0.403, 'tempo': 97.44, 'type': 'audio_features', 'id': '7eX5SypK35V8Y9d9pS6rWy', 'uri': 'spotify:track:7eX5SypK35V8Y9d9pS6rWy', 'track_href': 'https://api.spotify.com/v1/tracks/7eX5SypK35V8Y9d9pS6rWy', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7eX5SypK35V8Y9d9pS6rWy', 'duration_ms': 162040, 'time_signature': 4}, {'danceability': 0.63, 'energy': 0.908, 'key': 11, 'loudness': -2.42, 'mode': 1, 'speechiness': 0.0326, 'acousticness': 0.0238, 'instrumentalness': 0.592, 'liveness': 0.116, 'valence': 0.968, 'tempo': 120.522, 'type': 'audio_features', 'id': '4fPBB44eDH71YohayI4eKV', 'uri': 'spotify:track:4fPBB44eDH71YohayI4eKV', 'track_href': 'https://api.spotify.com/v1/tracks/4fPBB44eDH71YohayI4eKV', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4fPBB44eDH71YohayI4eKV', 'durat

In [5]:
embed_remover = re.compile(r'\D*')

def remove_song_anatomy_tags(l):
    l = l.split('\n')
    l = l[1::]
    edited = []

    last_line = l[-1]
    last_line = last_line.split(' ')
    last_line[-1] = re.findall(embed_remover, last_line[-1])[0]
    l[-1] = ' '.join(last_line)

    for i in l:
        if i == '':
            continue

        if i == ' ':
            continue

        edited.append(i)

    return ' '.join(edited)


def get_song_lyrics_and_features(track_id, track_name, artist_name, popularity=50, explicit=0):
    feature_list = []
    track_features = sp.audio_features(tracks = [track_id])[0]

    feature_names = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'duration_ms']

    for feature in feature_names:
        feature_list.append(track_features[feature])
        
    feature_list.append(popularity)
    feature_list.append(explicit)
    print(feature_list)

    
    max_tries = 4

    for i in range(max_tries):

        try:
            song = genius.search_song(track_name, artist_name)
            break
        except requests.exceptions.Timeout:
            print(f'{track_name} pooped itself: {i} tries')
            if i == max_tries - 1:
                song = 'blank'
                raise

    # print(song.lyrics)
    embeddings = embedder.encode(remove_song_anatomy_tags(song.lyrics), show_progress_bar=True)
    embeddings_proj = np.dot(R, embeddings)

    feature_list.extend(embeddings_proj)
    print(feature_list)
    return feature_list


# objs = list(bucket.objects.filter(Prefix=track_id))
#     if len(objs) > 0 and objs[0].key == track_id:
#         print(f"{track_name} is already in the S3 bucket.")
#         continue

In [8]:
playlist_id = 'https://open.spotify.com/playlist/2AXUGe93UP8oVj3F0a1Hoo?si=836ee38e473943a6'
results = sp.playlist_tracks(playlist_id)
# print(results)
for item in results['items'][:3]:
    track = item['track']
    track_id = track['id']
    track_name = track['name']
    artist_name = track['artists'][0]['name']
    popularity = track['popularity']
    explicit = int(track['explicit'])
    print(track_name, artist_name)
    get_song_lyrics_and_features(track_id, track_name, artist_name, popularity, explicit)
    print(track_id)



Don't Let the Green Grass Fool You Wilson Pickett
[0.573, 0.707, 6, -8.29, 0, 0.0632, 0.442, 0.000241, 0.123, 0.961, 152.467, 4, 169707, 51, 0]
Searching for "Don't Let the Green Grass Fool You" by Wilson Pickett...
Done.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0.573, 0.707, 6, -8.29, 0, 0.0632, 0.442, 0.000241, 0.123, 0.961, 152.467, 4, 169707, 51, 0, -4.43101237632808, -1.3624465109888164, 1.7886000610599309, 2.4334801231404457, 2.5731902978837504, 0.9116472704598317, 0.5386050733645735, 1.1442453832281665, -1.6419258944452841, -0.40489539001767416, -1.4054318218731816, -5.7961042523803, -6.676508637842356, -7.29755682948978, 0.4076802476496706, -1.3839950788670916, 1.4313023161942402, -3.384572124597999, -5.773655045663233, -0.5985818936859117, -6.153295052942207, 4.432238650479825, -0.6879306025188692, -5.4401336275893915, -2.8045419943604024]
79krwDoFzJ6dbNRwceAwgH
Pictures of You PREP
[0.919, 0.519, 2, -6.407, 1, 0.0537, 0.139, 2.67e-05, 0.166, 0.555, 115.992, 4, 235631, 0, 0]
Searching for "Pictures of You" by PREP...
Done.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0.919, 0.519, 2, -6.407, 1, 0.0537, 0.139, 2.67e-05, 0.166, 0.555, 115.992, 4, 235631, 0, 0, -1.6335635173852445, -0.9348303066209556, 2.7321617254753443, 5.936774990531057, 3.453786851430933, -5.150879706288572, -3.5402722080975035, 2.109513605553212, 5.139658582509666, -8.678709565149612, -0.10583859664801887, -1.1558839756243509, -2.3498391381588712, -4.111824575215946, -3.3858322366676985, 7.873967338923208, -3.690916586628629, 2.012634500411329, -2.2955384059575246, -1.782506487800636, -5.427592528787896, 2.9839141487648635, -0.24540869949188693, 2.873463547981641, 1.7247231273303767]
0fNY6p1S1PCwpnK9hlP4qz
I Can't Answer That PREP
[0.761, 0.457, 10, -7.232, 1, 0.0479, 0.0502, 0.000109, 0.106, 0.866, 153.008, 4, 246707, 31, 0]
Searching for "I Can't Answer That" by PREP...
Done.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0.761, 0.457, 10, -7.232, 1, 0.0479, 0.0502, 0.000109, 0.106, 0.866, 153.008, 4, 246707, 31, 0, -5.957649702339686, 1.7957610152647732, 3.4361783447211245, 5.177048339343599, -1.0346974649278269, -2.9145071332227848, -4.674081924847007, 6.535949933268457, 4.506398875871369, -5.054211235951135, -2.842479949470984, -3.496964941377114, -6.221338816272662, -5.954183997976298, -4.889607948185495, 1.6611747618060293, 0.16701277480897025, -4.627846796869395, -1.943536499630194, 3.177510752960368, -7.049686198735225, 2.902164584048769, -0.9488077252975273, 7.020878711788388, 1.7722166887563775]
3UkWgTbTdxYlGchZQEVu1A
